### Gaussian Copula MLE
create matricies for 

have 2 codes: one with gaussian, one with vtransform, both taking multiple days, both being copulas( ie divide buy marginals) once this is running, fit it on aust data with only the loc distance matrix 

work on getting a dist matrix for predictors

most important is vcopula ready with different days, and fit on aust data - first only with location matrix

In [1]:
from scipy.optimize import dual_annealing
import numpy as np
import itertools
from scipy.optimize import Bounds
from scipy.optimize import minimize
import scipy.stats as scs
from pynverse import inversefunc
from math import sqrt
from scipy.special import gamma
from scipy.special import kv 
from  geopy.geocoders import Nominatim
from itertools import combinations
from sklearn.gaussian_process.kernels import Matern


In [ ]:
class mvnorm():

    def __init__(self,mean=[0,0,0],L=np.array([[0,1,1],[1,0,1],[1,1,0]]),X=np.array([[0,2,3],[2,0,4],[3,4,0]])):
        self.mean=mean
        self.N=L.shape[0]
        self.L=L
        self.X=X
    def sim(self,sigma,p0,v0,p1,v1,n=10):
        cov_mat=Matern(length_scale=p0,nu=v0).__call__(self.L)* Matern(length_scale=p1,nu=v1).__call__(self.L) *np.power(sigma,2) 
        return [scs.multivariate_normal.rvs(mean=self.mean,cov=cov_mat) for i in range(n)]
    
    def nll(self,theta,data):
        #data is an array with multiple sets of joint realistaions
        #theta=[sigma,p0,v0,p1,v1]
        cov_mat=Matern(length_scale=theta[1],nu=theta[2]).__call__(self.L)* Matern(length_scale=theta[3],nu=theta[4]).__call__(self.L) *np.power(theta[0],2)      
        #and  theta[indexes]
        nll=0
        for obs in data:
            nll+= scs.multivariate_normal.logpdf(obs,mean=self.mean,cov=cov_mat) 
        return -nll

    #taking multiple days
    def __init__(self,L=np.array([[0,1,1],[1,0,1],[1,1,0]]),X=[np.array([[0,2,3],[2,0,4],[3,4,0]])]):
        '''
        L is location matrix: n x n
        X is predictor matrix, day dependent: [mat_1 , mat_2 , ..] m times n x n 
        '''
        self.N=L.shape[0]
        self.L=L
        self.X_all=X

    def sim(self,theta,n=10):
        cov_mat_all=[Matern(length_scale=theta[1],nu=theta[2]).__call__(self.L)* Matern(length_scale=theta[3],nu=theta[4]).__call__(X_current) *np.power(theta[0],2) for X_current in self.X_all] 
        return [scs.multivariate_normal.rvs(mean=self.mean,cov=cov_mat) for i in range(n)]
    
    def nll(self,theta,data):
        #data is an array with multiple sets of joint realistaions
        #theta=[sigma,p0,v0,p1,v1]
        cov_mat=Matern(length_scale=theta[1],nu=theta[2]).__call__(self.L)* Matern(length_scale=theta[3],nu=theta[4]).__call__(self.L) *np.power(theta[0],2)      
        #and  theta[indexes]
        nll=0
        for obs in data:
            nll+= scs.multivariate_normal.logpdf(obs,mean=self.mean,cov=cov_mat) 
        return -nll

In [66]:
    
class norm_cop():

    def __init__(self,L=np.array([[0,1,1],[1,0,1],[1,1,0]]),X=[np.array([[0,2,3],[2,0,4],[3,4,0]])]):
        '''
        L is location matrix: n x n
        X is predictor matrix, day dependent: [mat_1 , mat_2 , .., mat_days] days times n x n 
        '''
        self.N=L.shape[0]
        self.L=L
        self.X_all=X

    def sim(self,theta,draws=3,as_x=True):
        '''
        Returns: [ [m [draws of length n] for day t+1],..., [m [draws of length n] for day t+T_(depending on prediction matricies X_all)] ]
        So to get the samples from day t+1: sim[1-1]
        To get k th draw from samples at day t+T: sim[T-1][k-1]
        To get u for 3rd location of kth drw for day T: sim[T-1][k-1][3-1]'''
        cov_mat_all=[Matern(length_scale=theta[1],nu=theta[2]).__call__(self.L)* Matern(length_scale=theta[3],nu=theta[4]).__call__(X_current) *np.power(theta[0],2) for X_current in self.X_all] 
        if as_x==True:
            return [[scs.multivariate_normal.rvs(mean=np.zeros(self.N),cov=cov_mat) for i in range(draws) ] for cov_mat in cov_mat_all ]
        if as_x==False:
            Us=[[np.zeros(self.N) for i in range(draws)] for cov_mat in cov_mat_all ]
            print(Us)
            for day,cov_mat in enumerate(cov_mat_all):
                for i in range(draws):
                    Us[i][day]=[scs.multivariate_normal.cdf(y,mean=0,cov=cov_mat[idx,idx]) for idx,y in enumerate(scs.multivariate_normal.rvs(mean=np.zeros(self.N),cov=cov_mat))]
            return Us

    
    def nll(self,theta,data):
        #data is an array with multiple sets of joint realistaions
        #theta=[sigma,p0,v0,p1,v1]
        cov_mat=Matern(length_scale=theta[1],nu=theta[2]).__call__(self.L)* Matern(length_scale=theta[3],nu=theta[4]).__call__(self.L) *np.power(theta[0],2)      
        #and  theta[indexes]
        nll=0
        for obs in data:
            nll+= scs.multivariate_normal.logpdf(obs,mean=self.mean,cov=cov_mat) 
        return -nll

In [70]:
norm_cop(X=[np.array([[0,2,3],[2,0,i+1],[3,i+1,0]]) for i in range(1)]).sim([1,2,3,4,5,6,7],draws=2,as_x=True)

[[array([-1.67059506, -2.17823729, -1.67541737]),
  array([-0.48034577,  0.44067071,  0.23919535])]]

In [26]:
pog=scs.multivariate_normal.rvs(mean=[0,0],cov=np.matrix([[1,0],[0,1]]))
[scs.multivariate_normal.cdf(pog[k],mean=[0,0][k],cov=np.matrix([[1,0],[0,1]])[k,k]) for k in range(len(pog))]

[0.7884822354218189, 0.8502832158009465]

In [47]:
range(1)

range(0, 1)